# Sample Canon Analysis

## Select Image & Phantom and Parse

In [1]:
from pathlib import Path

from Parsers.canonBinParser import findPreset, getImage

In [2]:
# Inputs
imagePath = Path("/media/das/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/001/Preset_2/20220427104128_IQ.bin")
phantomPath = Path("/media/das/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/Phantom data/Preset_2/20220831121752_IQ.bin")

In [3]:
imPreset = findPreset(imagePath)
phantomPreset = findPreset(phantomPath)
assert imPreset == phantomPreset

In [4]:
imgDataStruct, imgInfoStruct, refDataStruct, refInfoStuct = getImage(
    f"{imagePath.name}", f"{imagePath.parent}", f"{phantomPath.name}", f"{phantomPath.parent}"
)

/home/das/QuantUS_Projs/QuantUS/Parsers/canonBinParser.py:84: RuntimeWarning: divide by zero encountered in log10
  bmode = 20*np.log10(abs(iq))


Preset 2 found!
Preset 2 found!


## ROI Selection (from GUI)

In [5]:
import pickle

pkl_name = Path("/media/das/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/Phantom data/Preset_2/test.pkl")

with open(pkl_name, "rb") as f:
    pkl_info = pickle.load(f)

In [6]:
assert pkl_info.imName == imagePath.name
assert pkl_info.phantomName == phantomPath.name

In [7]:
scSplineX = pkl_info.finalSplineX
scSplineY = pkl_info.finalSplineY

## Analysis Config (from GUI)

In [8]:
from pyQus.analysisObjects import Config

In [9]:
config = Config()
config.analysisFreqBand = [pkl_info.lowBandFreq, pkl_info.upBandFreq]
config.axialOverlap = pkl_info.axialOverlap
config.axWinSize = pkl_info.axialWinSize
config.lateralOverlap = pkl_info.lateralOverlap
config.latWinSize = pkl_info.lateralWinSize
config.samplingFrequency = pkl_info.samplingFreq
config.transducerFreqBand = [pkl_info.minFrequency, pkl_info.maxFrequency]
config.windowThresh = pkl_info.threshold/100

## Spectral Analysis

In [10]:
from pyQus.spectral import SpectralAnalysis
from pyQus.analysisObjects import UltrasoundImage

In [11]:
ultrasoundImage = UltrasoundImage()
ultrasoundImage.axialResRf = imgInfoStruct.depth / imgDataStruct.rf.shape[0]
ultrasoundImage.lateralResRf = ultrasoundImage.axialResRf * (
    imgDataStruct.rf.shape[0]/imgDataStruct.rf.shape[1]
) # placeholder
ultrasoundImage.bmode = imgDataStruct.scBmodeStruct.preScArr
ultrasoundImage.phantomRf = refDataStruct.rf
ultrasoundImage.rf = imgDataStruct.rf
ultrasoundImage.scBmode = imgDataStruct.scBmode
ultrasoundImage.xmap = imgDataStruct.scBmodeStruct.xmap
ultrasoundImage.ymap = imgDataStruct.scBmodeStruct.ymap

In [12]:
spectralAnalysis = SpectralAnalysis()
spectralAnalysis.ultrasoundImage = ultrasoundImage
spectralAnalysis.config = config
spectralAnalysis.scSplineX = scSplineX
spectralAnalysis.scSplineY = scSplineY

In [13]:
spectralAnalysis.splineToPreSc()
spectralAnalysis.generateRoiWindows()
spectralAnalysis.computeSpecWindows()

# Visualizations